In [1]:
import pickle 
from tlo.analysis.utils import (
    parse_log_file
)
import gzip
import os 
from pathlib import Path

# Unpickle files 

In [2]:
import tlo.logging.reader as _reader
import numpy as np

_original_get_log_dataframes = _reader.LogData.get_log_dataframes

def _patched_get_log_dataframes(self):
    output_logs = _original_get_log_dataframes(self)
    return output_logs

# Or patch the insert call directly by wrapping parse_log_file:
from tlo.analysis.utils import parse_log_file as _parse_log_file

def parse_log_file_safe(logfile):
    import pandas as pd
    outputs = _parse_log_file(logfile)
    return outputs

In [5]:
scenario_output_dir = '/Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z/2'

def create_pickles_for_single_draw(draw_folder_path, compressed_file_name_prefix=None):
    """Process a single draw folder directly."""
    def turn_log_into_pickles(logfile):
        pickle_path = logfile.parent / "tlo.methods.healthsystem.summary.pickle"
        if pickle_path.exists():
            print(f"Skipping {logfile} - pickle already exists")
            return
        
        print(f"Opening {logfile}")
        outputs = parse_log_file(logfile)
        for key, output in outputs.items():
            if key == "tlo.methods.healthsystem.summary":
                print(f" - Writing {key}.pickle")
                with open(logfile.parent / f"{key}.pickle", "wb") as f:
                    pickle.dump(output, f)
    
    def uncompress_and_save_logfile(compressed_file) -> Path:
        target = compressed_file.parent / str(compressed_file.name.replace('.gz', ''))
        try:
            with gzip.open(compressed_file, 'rb') as s:
                content = s.read()
            with open(target, "wb") as t:
                t.write(content)
            return target
        except gzip.BadGzipFile:
            print(f"File {compressed_file} is not gzipped, using as-is")
            return compressed_file
    
    # Directly process run folders within this draw folder
    run_folders = [f for f in os.scandir(draw_folder_path) if f.is_dir()]
    for run_folder in run_folders:
        if compressed_file_name_prefix is None:
            logfile = [x for x in os.listdir(run_folder) if x.endswith('.log')][0]
        else:
            compressed_file_name = [
                x for x in os.listdir(run_folder) if x.startswith(compressed_file_name_prefix)
            ][0]
            logfile = uncompress_and_save_logfile(Path(run_folder.path) / compressed_file_name)
        turn_log_into_pickles(logfile)

create_pickles_for_single_draw(draw_folder_path=scenario_output_dir, compressed_file_name_prefix='baseline_run_with_pop__')

Opening /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z/2/0/baseline_run_with_pop__2026-02-20T162205.log
Processing log file /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z/2/0/baseline_run_with_pop__2026-02-20T162205.log
Writing module-specific log files to /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z/2/0
Finished writing module-specific log files.
 - Writing tlo.methods.healthsystem.summary.pickle
Opening /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z/2/1/baseline_run_with_pop__2026-02-20T162200.log
Processing log file /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/baseline_run_with_pop-2026-02-20T161931Z/2/1/baseline_run_with_pop__2026-02-20T162200.log
Writing module-specific log files to /Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/base

# Delete files 

In [ ]:
from pathlib import Path

# Directory to clean
base_dir = Path('/Users/rem76/PycharmProjects/TLOmodel/outputs/rm916@ic.ac.uk/climate_scenario_runs_lhs_param_scan-2026-02-03T101537Z')

# Find all .log files that start with the specified prefix
all_log_files = list(base_dir.rglob('*.log'))
log_files = [f for f in all_log_files if f.name.startswith('climate_scenario_runs_lhs_param_scan__')]

print(f"Found {len(all_log_files)} total .log files")
print(f"Found {len(log_files)} files starting with 'climate_scenario_runs_lhs_param_scan__'")

# Show first 10 as preview
print("\nFirst 10 files to be deleted:")
for i, log_file in enumerate(log_files[:10]):
    print(f"  {i+1}. {log_file.relative_to(base_dir)}")

if len(log_files) > 10:
    print(f"  ... and {len(log_files) - 10} more")

# Ask for confirmation
print(f"\n{'='*60}")
response = input(f"Delete all {len(log_files)} 'climate_scenario_runs_lhs_param_scan__*.log' files? (yes/no): ")

if response.lower() == 'yes':
    deleted = 0
    errors = 0
    
    for log_file in log_files:
        try:
            log_file.unlink()
            deleted += 1
            if deleted % 50 == 0:
                print(f"Deleted {deleted}/{len(log_files)} files...")
        except Exception as e:
            print(f"Error deleting {log_file}: {e}")
            errors += 1
    
    print(f"\n{'='*60}")
    print(f"Deletion complete!")
    print(f"  Deleted: {deleted}")
    print(f"  Errors: {errors}")
    print(f"{'='*60}")
else:
    print("Deletion cancelled")